In [ ]:
# import os
# from dotenv import load_dotenv
# load_dotenv()

# api_key = os.getenv("mysportsfeeds_apikey")

# def snaps_api(api_key, season, week):
#     api_url = f"https://api.mysportsfeeds.com/v2.1/pull/nfl/{season[0]}/week/{week[0]}/player_gamelogs.json"

#     try:
#         response = requests.get(
#             url=f"{api_url}",
#             params={
#                 "fordate": "20211103"
#             },
#             headers={
#                 "Authorization": "Basic " + base64.b64encode("{}:{}".format(api_key, "MYSPORTSFEEDS").encode("utf-8")).decode("ascii")
#             }
#         )
#         print("Response HTTP Status Code: {status_code}".format(
#             status_code=response.status_code))
#     except requests.exceptions.RequestException:
#         print("HTTP Request failed")

#     r = response.content
#     soup = BeautifulSoup(r, "html.parser")
#     json = soup.text
#     df = pd.json_normalize(json["gamelogs"])
#     cols = list(df.columns)
#     not_drop = []
#     for c in cols:
#         if "game." in c or "player." in c or "team." in c or "snapCounts" in c:
#             not_drop.append(c)
            
#     snaps = df[not_drop]

#     time.sleep(5)
    
#     return snaps

# def all_snaps(seasons, weeks):
#     start = time.time()
#     final_df = pd.DataFrame()
#     for s in seasons:
#         for w in weeks:
#             snaps = snaps_api(s, w)
#             final_df = pd.concat([snaps, final_df])
            
#     print(time.time()-start)
#     return final_df

# seasons = ["2020-2021-regular", "2019-2020-regular", "2018-2019-regular"]
# weeks = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17"]
# all_snaps = all_snaps(seasons, weeks)
# all_snaps["stats.snapCounts.offenseSnaps"].unique()
# all_snaps["stats.snapCounts.defenseSnaps"].unique()
# all_snaps["stats.snapCounts.specialTeamSnaps"].unique()

In [1]:
import time
import pyodbc
import os
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sqlalchemy import inspect
from sqlalchemy.engine import URL
from dotenv import load_dotenv

__author__ = "Tim Cuddeback"
__copyright__ = "Copyright (c) 2021, Fantasy-Sidelines"
__credits__ = ["Tim Cuddeback", "Sportradar API", "Fantasy Sharks", "MySportsFeeds API"]
__license__ = "MIT License"
__version__ = "1.1.0"
__maintainer__ = "Tim Cuddeback"
__email__ = "cuddebtj@gmail.com"
__status__ = "Dev"

from sql_upload import *

load_dotenv()
sql_driver = os.getenv("sql_driver")
# check IP address for connection timeout errors
sql_server_16 = os.getenv("sql_server_16")
sql_server_17 = os.getenv("sql_server_17")
sql_database = os.getenv("sql_database")
sql_username = os.getenv("sql_username")
sql_password = os.getenv("sql_password")
api_key = os.getenv("sportradar_apikey")

year = [2020, 2019, 2018, 2017, 2016]

connection_string = ("DRIVER="+sql_driver
                     +";SERVER="+sql_server_16
                     +";DATABASE="+sql_database
                     +";UID="+sql_username
                     +";PWD="+sql_password)
# cxn = pyodbc.connect(connection_string)
connection_url = URL.create("mssql+pyodbc", 
                            query={"odbc_connect": connection_string})
engine = create_engine(connection_url)
conn = engine.connect()

inspector = inspect(engine)
print(inspector.get_table_names())

['IDCalendarTable', 'IDGameTable', 'IDPlayerTable', 'IDSeasonTable', 'IDTeamTable', 'IDVenueTable', 'IDWeekTable', 'playerPractice', 'playerSnaps', 'playerStats', 'schedule', 'teamStats', 'weeklyStats']


In [36]:
snaps_df = pd.read_sql_table("playerSnaps",  con=conn)
schedule = pd.read_sql_table("schedule", con=conn)
snaps_df = snaps_df.replace({"team": {
    "GNB": "GB", "JAX": "JAC", "KAN": "KC", 
    "LAR": "LA", "LVR": "LV", "NOR": "NO", 
    "NWE": "NE", "SDG": "SD", "SFO": "SF", 
    "TAM": "TB"
}, "opp": {
    "GNB": "GB", "JAX": "JAC", "KAN": "KC", 
    "LAR": "LA", "LVR": "LV", "NOR": "NO", 
    "NWE": "NE", "SDG": "SD", "SFO": "SF", 
    "TAM": "TB"
}})
snaps_df["home.away"] = snaps_df["home.away"].replace({"@": "away", "": "home", np.nan: "home"})
snaps_df[
    ["off.snaps",
    "def.snaps",
    "st.snaps"]
] = snaps_df [
    ["off.snaps",
    "def.snaps",
    "st.snaps"]
].fillna(0)
snaps_df[
    ["off.snaps",
    "def.snaps",
    "st.snaps"]
] = snaps_df [
    ["off.snaps",
    "def.snaps",
    "st.snaps"]
].replace("", 0)
snaps_df[
    ["off.snaps", 
    "def.snaps", 
    "st.snaps"]
] = snaps_df[
    ["off.snaps", 
    "def.snaps", 
    "st.snaps"]
].astype(int)
snaps_df["season"] = snaps_df["season"].astype(str)

replace = [" III", " II", " Sr.", " Jr.", " Jr", " Sr"]
snaps_df["player_name"] = snaps_df["player_name"].str.replace("|".join([re.escape(s) for s in replace]), "", regex=True)
snaps_df = snaps_df.drop_duplicates()
snaps_df = snaps_df.merge(schedule,
                         how="left",
                         left_on=["team", "home.away", "opp", "season"],
                         right_on=["game.team.alias", "home.away", "game.opp.alias", "season.year"],
                         suffixes=(None, ".merged"))
snaps_df = snaps_df[
    [
        "player_url", "player_name", "game.team.alias", "game.team.id",
        "home.away", "game.opp.alias", "game.opp.id", "game.id", "game.scheduled",
        "season.id", "season.year", "week.id", "week.sequence",
        "off.snaps", "def.snaps", "st.snaps"
    ]
]
snaps_df.to_sql("playerSnaps", engine, if_exists="replace", index=False, schema="dbo", chunksize=500)

In [ ]:
snaps(2016, 2020, engine)

Done: players_df
Time:  4.075072236855825  min
